# Лабораторная работа 9
# Создание текстового файла большого объема с помощью генераторной функции


## Функция randint из модуля random

In [1]:
from random import randint

In [2]:
?randint

Signature: randint(a, b)
Docstring:
Return random integer in range [a, b], including both end points.
        
File:      c:\users\amogus\appdata\local\programs\python\python311\lib\random.py
Type:      method

In [3]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
f'Код символа: {(c:=randint(*latin_code))}, символ: {chr(c)}'

'Код символа: 114, символ: r'

## Функция process_time из модуля time


In [4]:
from time import process_time


In [5]:
?process_time

Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method

In [6]:
start = process_time()
[x**2 for x in range(1_000_000)]
end = process_time()
print(f'{end - start} секунд')


0.0625 секунд


## Задание 9.1. Запись данных в текстовый файл


In [7]:
file_name = 'test.txt'
f = open(file_name, 'w')

In [8]:
lines_list = [f'{x}\n' for x in 'test']
lines_list


['t\n', 'e\n', 's\n', 't\n']

In [9]:
f.writelines(lines_list)


In [10]:
f.close()


In [11]:
with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


In [12]:
lines_gen_expr = (f'{x}\n' for x in range(2**20))

def lines_gen_fun(number_lines=2**20):
    yield from (f'{x}\n' for x in range(number_lines))
    
with open(file_name,'w') as f:
    f.writelines(lines_gen_expr)
    f.writelines(lines_gen_fun())


In [13]:
def lines_gen_fun(number_lines = 2**20):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
    print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
    f.writelines(lines_gen_fun())


6.940362930297852 Mb


In [14]:
len('f'), len('f'.encode('utf8')), len('ц'), len('ц'.encode('utf8'))

(1, 1, 1, 2)

In [15]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun())
    end = process_time()
    print(end - start, 'секунд')

6.940362930297852 Mb
0.59375 секунд


In [16]:
def lines_gen_fun(number_lines=2**20, status=False):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
        if status:
            status_number = x/number_lines*100
            print(f'\r{int(status_number)}%', end='', flush=True)
    print(f'\n {file_size/1024**2} Mb') 


In [17]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun(3*10**3, status=True))
    end = process_time()
    print(f'{end - start} секунд')

99%
 0.013246536254882812 Mb
1.140625 секунд


## Задание 9.2. Генерация строк


In [18]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
number_words = (10, 20)
number_symbols = (5, 10)

In [19]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
word

'zlxewqwuw'

In [20]:
number_code = (ord('0'), ord('9'))
cyrillic_code = (ord('а'),ord('я'))


In [21]:

result = ''
for rand in range(randint(1,10)):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
    result += word
    result += ' '
    
print(result)

result = ''
for rand in range(randint(1,10)):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*cyrillic_code)) for i in range(len_word)])
    result += word
    result += ' '
    
print(result)

result = ''
for rand in range(randint(1,10)):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*number_code)) for i in range(len_word)])
    result += word
    result += ' '
    
print(result)

dgifngnd gdbbb mlvxzhu liwhyz vptrhco vfiqx sgfwnfu sgkrwcqn 
дыляьчдг кгихччйс рмрикрль шуытуэзцкт зшлйы еуитбш тэгттц ымпхц еаьвь лгфдзяпзя 
886187555 8512099229 9733202743 86628428 


In [22]:
def lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols):
    file_size_count = 0
    target_bytes = file_size * 2**20
    while file_size_count < target_bytes:
        result = ''

        for _ in range(randint(*number_words)):

            len_word = randint(*number_symbols)

            word = ''.join([chr(randint(*type_symbols)) for _ in range(len_word)])
            word_bytes = word.encode('utf-8')

            if file_size_count + len(word_bytes) + 1 > target_bytes:
                break

            result += word + ' '
            file_size_count += len(word_bytes) + 1
        
        if not result:
            break


        result = result.rstrip() + '\n'
        file_size_count += 1

        yield result

In [23]:
with open('numeric.txt','w') as f:
    f.writelines(lines_gen_fun_v2(1,number_code,(1,5),(1,15)))

In [24]:
with open('cyrillic.txt','w') as f:
    f.writelines(lines_gen_fun_v2(1,cyrillic_code,(1,10),(1,10)))

In [25]:
with open('latinic.txt','w') as f:
    f.writelines(lines_gen_fun_v2(1,latin_code,(1,10),(1,10)))

In [26]:
def progress(gen_func):
    def wrapper(file_size,*args,**kwargs):
        target_bytes = file_size * 2**20
        x = 0
        for i in gen_func(file_size,*args,**kwargs):
            word_bytes = i.encode('utf-8')
            x += len(word_bytes)
            status_number = x/target_bytes*100
            print(f'\r{int(status_number)}%', end='', flush=True)
            yield i
    return wrapper


def file_gen(file_name: str, file_size: int, type_symbols: str='latin',number_words: (int,int)=(10,20), number_symbols: (int,int)=(5,10)):
    start = process_time()
    progress_lines = progress(lines_gen_fun_v2)
    if type_symbols == 'latin':
        symbols_tuple = latin_code
    elif type_symbols == 'cyrillic':
        symbols_tuple = cyrillic_code
    else:
        symbols_tuple = number_code
    with open(file_name,'w') as f:
        f.writelines(progress_lines(file_size,symbols_tuple,number_words,number_symbols))
    print ()
    end = process_time()
    print (end-start)
    

In [27]:
file_gen('aboba.txt',2)

99%
8.0625


In [28]:
file_gen('aboba2.txt',2,'numeric',(4,6),(5,6))

97%
25.71875


In [29]:
file_gen('aboba3.txt',2,'cyrillic',(1,10),(6,10))

98%
9.265625
